In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable as V
from lib.Exercise1_1 import LQRSolver
from torch.utils.data import TensorDataset, DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import time 

Proj_dtype = torch.double
Proj_device = 'cpu' 
class DGMNN(nn.Module):
    def __init__(self):
        super(DGMNN, self).__init__()
        self.layer1 = nn.Linear(3, 100)  
        self.layer2 = nn.Linear(100, 100)
        self.layer3 = nn.Linear(100, 100)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.output = nn.Linear(100, 1)
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.tanh(self.layer2(x))
        x = self.relu(self.layer3(x))

        return self.output(x)



In [3]:
class DGMNN2(nn.Module):
    def __init__(self):
        super(DGMNN2, self).__init__()
        self.layer1 = nn.Linear(3, 100)
        self.layer2 = nn.Linear(100, 100)
        self.layer3 = nn.Linear(100, 100)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.output = nn.Linear(100, 1)
        
    def forward(self, x):

        out1 = self.relu(self.layer1(x))
        identity = out1
        out2 = self.tanh(self.layer2(out1)+identity)
        identity = out2
        out3 = self.relu(self.layer3(out2)+identity)
        return self.output(out3)


In [4]:
def get_hessian(grad,x):
    Hessian = torch.tensor([], device = Proj_device)
    
    for i in range(len(x)):
        hessian = torch.tensor([], device = Proj_device)
        for j in range(len(grad[i])):
            u_xxi = torch.autograd.grad(grad[i][j], x, grad_outputs=torch.ones_like(grad[i][j]), retain_graph=True,create_graph=True, allow_unused=True)[0]           
            hessian = torch.cat((hessian, u_xxi[i].unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)),dim = 0)
        #print(Hessian)
    return Hessian

def get_hessian_(model,t,x):
    Hessian = torch.tensor([], device = Proj_device)
    for i in range(len(t)):
        x_i = V(x[i],requires_grad=True)
        input = torch.cat(((t[i]).unsqueeze(0), x_i),dim=0)
        u_in = model(input)
        grad = torch.autograd.grad(u_in, x_i, grad_outputs=torch.ones_like(u_in), create_graph=True, retain_graph=True)[0]
        hessian = torch.tensor([], device = Proj_device)
        for j in range(len(grad)):
            u_xxi = torch.autograd.grad(grad[j], x_i, grad_outputs=torch.ones_like(grad[j]), retain_graph=True,create_graph=True, allow_unused=True)[0]           
            hessian = torch.cat((hessian, u_xxi.unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)),dim = 0)
    return Hessian

def pde_residual(model, t, x):
    
    input = torch.cat((t.unsqueeze(1), x),dim=1)
    
    u = model(input)

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]

    u_xx = get_hessian(u_x,x)

#    u_xx = get_hessian_(model,t,x)
 
    residual = u_t + 0.5 * torch.einsum('bii->b', sigma @ sigma.transpose(1,2) @ u_xx) + (u_x.unsqueeze(1) @ (H @ x.unsqueeze(1).transpose(1,2)) + u_x.unsqueeze(1) @ M @ alpha + x.unsqueeze(1) @ C @ x.unsqueeze(1).transpose(1,2) + alpha.transpose(1,2) @ D @ alpha).squeeze()
    return residual

def boundary_condition(model,t, x):

    
    T_input = T * torch.ones_like(t)

    input = torch.cat((T_input.unsqueeze(1), x),dim=1)
    u = model(input)

    return u - (x.unsqueeze(1) @ R @ x.unsqueeze(1).transpose(1,2)).squeeze()

def total_residual(model, t, x):
    
    residual_loss = pde_residual(model, t, x).pow(2).mean()
    boundary_loss = boundary_condition(model,t,x).pow(2).mean()
    
    return residual_loss + boundary_loss


In [5]:
def new_data(num_samples):
    #num_samples = 10000
    t_samples = T * torch.rand(num_samples, dtype=Proj_dtype, device = Proj_device, requires_grad=True)
    x_ends = torch.tensor([-3,3], dtype = Proj_dtype)
    x_samples = x_ends[0] + (x_ends[1]- x_ends[0]) * torch.rand(num_samples , 2, dtype=Proj_dtype, device = Proj_device, requires_grad=True)
    return t_samples,x_samples

In [6]:
# Define matrices for LQR problem

H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=Proj_dtype, device = Proj_device)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=Proj_dtype, device = Proj_device)
sigma = torch.tensor([[[0.08],[0.11]]], dtype=Proj_dtype, device = Proj_device)
alpha = torch.tensor([[[1],[1]]], dtype=Proj_dtype, device = Proj_device)
C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=Proj_dtype, device = Proj_device)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=Proj_dtype, device = Proj_device)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=Proj_dtype, device = Proj_device)
T = torch.tensor(1.0, dtype=Proj_dtype, device = Proj_device)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")


In [9]:
samplesize = [100,1000,5000]


for sz in samplesize:


    model_DGM = DGMNN2().double()
    # stat_dict = torch.load('model2_DGM_state_dict.pt', map_location=torch.device('cpu'))
    # model_DGM.load_state_dict(stat_dict)
    #model_DGM = DGMNN().float().to(Proj_device)
    # Prepare for training
    optimizer_DGM = torch.optim.Adam(model_DGM.parameters(), lr=0.01)
    scheduler_DGM = lr_scheduler.ExponentialLR(optimizer_DGM, gamma=0.9)


    epoch_losses = []
    # 新建文本文件,f'epochloss{%d}'sz
    # 新建文本文件用于存储epoch的损失
    loss_filename = f'epoch_loss_{sz}.txt'
    with open(loss_filename, 'w') as file:
        file.write('Epoch,Loss\n')

    Batch_size = 5
    epochs = 40

    for batch in range(Batch_size):
    
        print(f'Batch {batch+1}/{Batch_size}'+'\n')
        
        t_data,x_data = new_data(sz)
        dataset = TensorDataset(t_data,x_data)
        dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

        for epoch in range(epochs):

            model_DGM.train()
            total_loss = 0
            
            for batch_idx, (t_data_,x_data_) in enumerate(dataloader):
                optimizer_DGM.zero_grad()
                t_v = V(t_data_,requires_grad=True)
                x_v = V(x_data_,requires_grad=True)
                loss = total_residual(model_DGM, t_v, x_v) 
                loss.backward(retain_graph=False)
                #loss.backward(retain_graph=True)
                optimizer_DGM.step()
                total_loss += loss.item()
            epoch_losses.append(total_loss / len(dataloader))
            
            scheduler_DGM.step()
            if epoch == 0:
                print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')
            if(epoch+1)% 10 == 0:
                torch.save(model_DGM.state_dict(), f'model2_DGM_state_dict_{sz}.pt')
            if (epoch+1) % 5 == 0:
                print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')

        print('\n')
        # sdhsl
        # 在每个批次结束后，以追加模式将epoch损失数据添加到文件
        with open(loss_filename, 'a') as file:
            for i, loss in enumerate(epoch_losses, 1):
                file.write(f'{i},{loss}\n')
    model_DGM.eval()

Batch 1/5

Epoch 1/100 	 Loss: 158.15209147224454
Epoch 5/100 	 Loss: 87.62574294569565
Epoch 10/100 	 Loss: 49.41909996728251
Epoch 15/100 	 Loss: 37.939673845038186
Epoch 20/100 	 Loss: 33.003923233109106
Epoch 25/100 	 Loss: 30.36266420904794
Epoch 30/100 	 Loss: 29.37974029015094
Epoch 35/100 	 Loss: 28.71237033580825
Epoch 40/100 	 Loss: 28.354826613375174
Epoch 45/100 	 Loss: 27.986046474304594
Epoch 50/100 	 Loss: 27.75215596250623
Epoch 55/100 	 Loss: 27.678528613434686
Epoch 60/100 	 Loss: 27.60964817811614
Epoch 65/100 	 Loss: 27.546015611409853
Epoch 70/100 	 Loss: 27.517403580664556
Epoch 75/100 	 Loss: 27.524985587174132
Epoch 80/100 	 Loss: 27.51666185973881
Epoch 85/100 	 Loss: 27.5106306775058
Epoch 90/100 	 Loss: 27.50703764220195
Epoch 95/100 	 Loss: 27.5017513333675
Epoch 100/100 	 Loss: 27.500476665684374


Batch 2/5

Epoch 1/100 	 Loss: 36.57270957036067
Epoch 5/100 	 Loss: 36.57119730906974
Epoch 10/100 	 Loss: 36.56939568767366
Epoch 15/100 	 Loss: 36.60757767531

KeyboardInterrupt: 

In [23]:
model_DGM = DGMNN2().double()
stat_dict = torch.load('model2_DGM_state_dict_5000.pt', map_location=torch.device('cpu'))
model_DGM.load_state_dict(stat_dict)
model_DGM.eval()

DGMNN2(
  (layer1): Linear(in_features=3, out_features=100, bias=True)
  (layer2): Linear(in_features=100, out_features=100, bias=True)
  (layer3): Linear(in_features=100, out_features=100, bias=True)
  (tanh): Tanh()
  (relu): ReLU()
  (output): Linear(in_features=100, out_features=1, bias=True)
)

In [15]:
load_interval_setting = torch.load('Exercise3_MC/value_numerical/3x3/'+'interval_setting.pt')
t_ends = load_interval_setting['t_ends']
t_num = load_interval_setting['t_num']
x_ends = load_interval_setting['x_ends']
x_num = load_interval_setting['x_num']

# Establish meshgrid structure from setting.

t_batch_i = torch.linspace(t_ends[0],t_ends[1],t_num,dtype=torch.double)
t_batch = t_batch_i.repeat_interleave(x_num[0]*x_num[1])

x1 = torch.linspace(x_ends[0][0],x_ends[0][1],x_num[0],dtype=torch.double)
x2 = torch.linspace(x_ends[1][0],x_ends[1][1],x_num[1],dtype=torch.double)

x_batch = torch.cartesian_prod(x1, x2).unsqueeze(1).repeat(t_num, 1, 1)

x_batch_i = torch.cartesian_prod(x1, x2).unsqueeze(1)

X1 = x_batch_i[:, 0, 0].view(x_num[0], x_num[1])
X2 = x_batch_i[:, 0, 1].view(x_num[0], x_num[1])


In [16]:

file_path_MC_FSS = 'Exercise3_MC'+ '/' + f'value_MC/{x_num[0]}x{x_num[1]}/FSS_1e5'

FSS_VTSN = [int(x) for x in[5e3]]

MSE_FSS_VTSN = []
J = []
for i in FSS_VTSN:
    if i == 1:
        trvlthg = ''
    else:
        trvlthg = 's'
    path_FSS_VTSN_i = f"{file_path_MC_FSS}/{i}_step{trvlthg}"

    J_load_test = torch.load(path_FSS_VTSN_i + '/value_MC.pt')
    
    J_processed,_ = torch.min(J_load_test,dim = 1)

    J.append(J_processed)

In [17]:
J

[tensor([1.3017e+04, 2.3263e+03, 5.1811e+02, 4.4414e+03, 3.0043e+00, 4.4438e+03,
         5.1807e+02, 2.3284e+03, 1.3023e+04, 6.4233e+03, 1.1970e+03, 2.8020e+02,
         2.1580e+03, 1.9030e+00, 2.1573e+03, 2.8078e+02, 1.1973e+03, 6.4229e+03,
         3.1621e+03, 6.2128e+02, 1.6471e+02, 1.0424e+03, 1.2883e+00, 1.0420e+03,
         1.6474e+02, 6.2202e+02, 3.1622e+03, 1.5508e+03, 3.2614e+02, 1.0586e+02,
         5.0345e+02, 9.3517e-01, 5.0310e+02, 1.0581e+02, 3.2611e+02, 1.5509e+03,
         7.5768e+02, 1.7330e+02, 7.3826e+01, 2.4318e+02, 7.0335e-01, 2.4313e+02,
         7.3837e+01, 1.7329e+02, 7.5778e+02, 3.6838e+02, 9.3428e+01, 5.4422e+01,
         1.1848e+02, 5.2839e-01, 1.1849e+02, 5.4438e+01, 9.3441e+01, 3.6838e+02,
         1.7743e+02, 5.0974e+01, 4.1271e+01, 5.8761e+01, 3.8132e-01, 5.8735e+01,
         4.1279e+01, 5.0971e+01, 1.7738e+02, 8.4289e+01, 2.8008e+01, 3.1423e+01,
         3.0081e+01, 2.4854e-01, 3.0077e+01, 3.1427e+01, 2.8018e+01, 8.4262e+01,
         3.9005e+01, 1.5326e

AttributeError: 'DGMNN' object has no attribute 'dtype'

In [24]:
t_data = t_batch
x_data = x_batch.squeeze(1)
model_DGM(torch.cat((t_data.unsqueeze(1), x_data),dim=1)).squeeze()

tensor([12.6758, 13.5872,  6.5018, 16.9937,  8.9648,  9.4376, 13.2076, 10.8356,
         6.3437, 11.8149, 12.7126,  6.3833, 15.6533,  8.4713,  9.0772, 12.0247,
        10.0317,  6.0313, 10.9536, 11.8555,  6.1758, 14.2685,  8.0615,  8.7302,
        10.7794,  9.2463,  5.6596, 10.0726, 10.9751,  5.8491, 12.9659,  7.6795,
         8.4204,  9.5808,  8.5274,  5.2712,  9.0934, 10.1261,  5.4429, 11.6663,
         7.3249,  8.0785,  8.3503,  7.8867,  4.9247,  8.0682,  9.2537,  4.9394,
        10.3390,  7.0110,  7.6229,  7.2063,  7.2879,  4.5727,  7.0748,  8.3570,
         4.4035,  9.0457,  6.7364,  7.1673,  6.1127,  6.6515,  4.1690,  6.1062,
         7.4421,  3.8397,  7.8730,  6.4482,  6.6214,  5.0208,  6.0929,  3.8032,
         5.1497,  6.4806,  3.2508,  6.7754,  6.1572,  6.0658,  4.0177,  5.4764,
         3.4674], dtype=torch.float64, grad_fn=<SqueezeBackward0>)